<a href="https://colab.research.google.com/github/JanMarcelKezmann/Residual-Network-Architectures/blob/master/Residual_Network_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Input, Add, Dense, Flatten, Activation, BatchNormalization, Conv2D, ZeroPadding2D, AveragePooling2D
from tensorflow.keras.models import Model, save_model, load_model
from tensorflow.keras.initializers import glorot_uniform
import tensorflow.keras.backend as K

K.set_image_data_format("channels_last")
K.set_learning_phase(1)

from IPython.display import SVG
from matplotlib.pyplot import imshow
%matplotlib inline

# Create all Building Blocks

## Identity Block

In [0]:
def identity_block(X, f, filters, stage, block):
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    # Filters
    F1, F2, F3 = filters

    # Save the input value as Shortcut
    X_shortcut = X

    # Main Path
    # First Component of Main Path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(1, 1), padding="valid", name=conv_name_base + "2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)

    # Second Component of Main Path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding="same", name=conv_name_base + "2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)

    # Third Component of Main Path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="valid", name=conv_name_base + "2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2c")(X)
    X = Activation("relu")(X)

    # Final Step: Add the Shortcut to the Main Path
    X = Add()([X, X_shortcut])
    X = Activation("relu")(X)

    return X

## Convolutional Block

In [0]:
def convolutional_block(X, f, filters, stage, block, strides=2):
    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    # Filters
    F1, F2, F3 = filters

    # Save input value as shortcut
    X_shortcut = X

    # Main Path
    # First Component of Main Path
    X = Conv2D(filters=F1, kernel_size=(1, 1), strides=(strides, strides), padding="valid", name=conv_name_base + "2a", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2a")(X)
    X = Activation("relu")(X)

    # Second Component of Main Path
    X = Conv2D(filters=F2, kernel_size=(f, f), strides=(1, 1), padding="same", name=conv_name_base + "2b", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2b")(X)
    X = Activation("relu")(X)

    # Third Component of Main Path
    X = Conv2D(filters=F3, kernel_size=(1, 1), strides=(1, 1), padding="valid", name=conv_name_base + "2c", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name=bn_name_base + "2c")(X)
    X = Activation("relu")(X)

    # Shortcut Path
    X_shortcut = Conv2D(filters=F3, kernel_size=(1, 1), strides=(strides, strides), padding="valid", name=conv_name_base + "1", kernel_initializer=glorot_uniform(seed=0))(X_shortcut)
    X_shortcut = BatchNormalization(axis=3, name=bn_name_base + "1")(X_shortcut)

    # Final Step: Add shortcut path to main path
    X = Add()([X_shortcut, X])
    X = Activation("relu")(X)

    return X

In [0]:
# tf.reset_default_graph()

# with tf.Session() as sess:
#     np.random.seed(1)
#     A_prev = tf.placeholder("float", [3, 4, 4, 6])
#     X = np.random.randn(3, 4, 4, 6)
#     A = identity_block(A_prev, f=3, filters=[2, 4, 6], stage=1, block="a")
#     sess.run(tf.global_variables_initializer())

#     out = sess.run([A], feed_dict={A_prev: X, K.learning_phase(): 0})
#     print("out = " + str(out[0][0][0][0]))
#     print(np.size(out))

## ResNet 101

In [0]:
def ResNet101(input_shape=(224, 224, 3), classes=100):
    # Define Input as tensor with input_shape as shape
    # Default shape: (224x224x3)
    X_input = Input(input_shape)

    # ZeroPadding2D(((7 - 1) / 2, (7 - 1) / 2))(X_input)
    # Output shape: (230x230x3)
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    # Output shape after Conv2D: (112x112x64)
    X = Conv2D(64, kernel_size=(7, 7), strides=(2, 2), name="conv1", kernel_initializer=glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis=3, name="bn_conv1")(X)
    X = Activation("relu")(X)
    
    # Output shape after MaxPooling2D: (56x56x64)
    X = MaxPooling2D((3, 3), strides=(2, 2), padding="same")(X)

    # Stage 2
    # Output shape after stage 2: (56x56x256)
    X = convolutional_block(X, f=3, filters=[64, 64, 256], stage=2, block="a", strides=1)
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block="b")
    X = identity_block(X, f=3, filters=[64, 64, 256], stage=2, block="c")
    
    # Stage 3
    # Output shape after stage 3: (28x28x512)
    X = convolutional_block(X, f=4, filters=[128, 128, 512], stage=3, block="a", strides=2)
    X = identity_block(X, f=4, filters=[128, 128, 512], stage=3, block="b")
    X = identity_block(X, f=4, filters=[128, 128, 512], stage=3, block="c")
    X = identity_block(X, f=4, filters=[128, 128, 512], stage=3, block="d")

    # Stage 4
    # Output shape after stage 4: (14x14x1024)
    X = convolutional_block(X, f=6, filters=[256, 256, 1024], stage=4, block="a", strides=2)
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="b")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="c")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="d")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="e")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="f")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="g")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="h")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="i")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="j")

    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="k")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="l")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="m")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="n")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="o")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="p")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="q")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="r")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="s")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="t")

    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="u")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="v")
    X = identity_block(X, f=6, filters=[256, 256, 1024], stage=4, block="w")

    # Stage 5
    # Output shape after stage 5: (7x7x2048)
    X = convolutional_block(X, f=3, filters=[512, 512, 2048], stage=5, block="a", strides=2)
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block="b")
    X = identity_block(X, f=3, filters=[512, 512, 2048], stage=5, block="c")

    # Final Stage
    # Output shape after Dense Layer: (classes)
#     X = AveragePooling2D((2, 2), name="avg_pool")(X)
    X = Flatten()(X)
    X = Dense(classes, activation="softmax", name="fc" + str(classes), kernel_initializer=glorot_uniform(seed=0))(X)

    # Create Model
    model = Model(inputs=X_input, outputs=X, name="ResNet101")

    return model